In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt
import os
from themachinethatgoesping.echosounders.simrad import FileSimradRaw,FileSimradRaw_mapped,SimradDatagram_type_from_string,datagram_type_to_string,t_SimradDatagramIdentifier
import themachinethatgoesping.echosounders.simrad as simrad
import themachinethatgoesping.echosounders.simrad.datagrams as dg
import themachinethatgoesping.tools.progressbars as prg
import themachinethatgoesping.tools as tools
import themachinethatgoesping.navigation.nmea_0183 as NMEA
import numpy
import os, sys
from time import time
#from tqdm.auto import tqdm
from tqdm import tqdm
from collections import defaultdict
import numpy as np

#load files
folders=[]
with open('test_data.txt','r') as ifi:
    for line in ifi:
        print("Folder:", line)
        folders.append(line)

if not len(folders):
    print("Please add test_data folders to 'test_data.txt'")

files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
            
files.sort()
#files
file_name = files[0]

In [ ]:

#fm = FileSimradRaw_mapped(files[0:20])
fm = FileSimradRaw_mapped(files)
print(fm)

In [ ]:
nav = fm.navigation_interface.per_file(0).datagrams("NME0")[610].decode()
print(nav)

In [ ]:
conf = fm.configuration_interface.per_file(0).datagrams("XML0")[0].decode()

In [ ]:
print(conf.ConfiguredSensors[6].Telegrams[0])

In [ ]:
print(conf.get_sensor_configuration())

In [ ]:
%%timeit -n1 -r10
d = fm.datagram_interface.datagrams()

In [ ]:
d = fm.datagram_interface.datagrams()
print(d)

In [ ]:
print(d)

In [ ]:
%%timeit -n1 -r10
d = fm.pings()

In [ ]:
d = fm.datagram_interface.datagrams()

In [ ]:
%%timeit -n1 -r10
d.get_sorted_by_time()

In [ ]:
d[0]

In [ ]:
#%%timeit -n1 -r1
if False:
    conv_factor = 10. * np.log10(2.0) / 256.

    data=[]
    for raw3 in tqdm(fm.datagram_interface.datagrams("RAW3")):
        if raw3.data_type == raw3.data_type.ComplexFloat32:
            a = raw3.sample_data.complex_samples
            b = np.sum(a,axis=1)
            b = np.sum(b*b,axis=1)
            data.append(b)
        elif raw3.data_type == raw3.data_type.PowerAndAngle:
            a = raw3.sample_data.power[0].astype(np.float32) * conv_factor
            data.append(a)

In [ ]:
#%%timeit -n1 -r1

steps=1
min_=0
max_=-1

data=[]
angle1=[]
angle2=[]
#for raw3 in tqdm(fm.datagram_interface("RAW3",0,-1,steps)):
for raw3 in tqdm(fm.datagram_interface.datagrams("RAW3")[min_*steps:max_*steps:20]):
    if raw3.data_type == raw3.data_type.ComplexFloat32:
        data.append(10*np.log10(raw3.sample_data.get_power()))
        pass
    elif raw3.data_type == raw3.data_type.PowerAndAngle:
        data.append(raw3.sample_data.get_power())
        angle = raw3.sample_data.get_angle()
        angle1.append(angle[:,0])
        angle2.append(angle[:,1])
        
        pass

In [ ]:
#z_max = 3500

def to_np(data,z_max=5000):
    m=0
    for d in tqdm(data):
        if len(d) > m:
            m = len(d)

    m = z_max

    arr = np.empty((len(data),m))
    arr.fill(np.nan)

    for d in tqdm(range(0,len(data))):
        z = min(z_max,len(data[d]))
        arr[d][:z] = data[d][:z]

    arr = 10*np.log10(arr + 0.0000000000001)
    print(arr.shape)
    return arr

arr1 = to_np(angle1)
arr2 = to_np(angle2)
arr = to_np(data)

In [ ]:
plt.close("echogram")
fig = plt.figure("echogram",figsize=(12,4))
ax = fig.subplots()
mapped = ax.imshow(arr.transpose(),aspect='auto',cmap='YlGnBu_r', vmin = -150, vmax=-30)
fig.colorbar(mapped)
fig.tight_layout()

In [ ]:
plt.close("angle1")
fig = plt.figure("angle1",figsize=(12,4))
ax = fig.subplots()#!/bin/xonsh

mapped = ax.imshow(arr1.transpose(),aspect='auto',cmap='YlGnBu_r')
fig.colorbar(mapped)
fig.tight_layout()

In [ ]:
plt.close("angle1")
fig = plt.figure("angle1",figsize=(12,4))
ax = fig.subplots()
mapped = ax.imshow(arr2.transpose(),aspect='auto',cmap='YlGnBu_r')
fig.colorbar(mapped)
fig.tight_layout()